<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/image_type_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Tensorflow MobileNetSSD v2 and Inception v3 models to classify maps, phylogenies, illustrations, and herbarium sheets from EOL images
---
*Last Updated 10 November 2022*   
Train [MobileNet SSD v2](https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4) and [Inception v3](https://tfhub.dev/google/imagenet/inception_v3/classification/4) via fine-tuning (unfreezing lower layers) to classify EOL image types as maps, phylogenies, illustrations, or herbarium sheets. The training dataset consists of image bundles from a variety of sources. Maps and phylogenies are from Wikimedia Commons, herbarium sheets are from NMNH Botany on EOL, botanical illustrations are from Flickr BHL, and zoological images are from EOL. Classifications will be used to generate image tags to improve searchability of EOLv3 images.

Training images were downloaded to Google Drive and processed using [image_type_preprocessing.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/image_type_preprocessing.ipynb). Manual inspection of images was used to determine and apply exclusion criteria for each training class.

***Models were trained in Python 2 and TF 1 in October 2020: MobileNet SSD v2 was trained for 3 hours to 30 epochs with Batch Size=16, Lr=0.00001, Dropout=0.3, epsilon=1e-7, Adam optimizer. Final validation accuracy = 0.90. Inception v3 was trained for 3.5 hours to 30 epochs with Batch Size=16, Lr=0.0001, Dropout=0.2, epsilon=1, Adam optimizer. Final validation accuracy = 0.89.***

Notes:
* Run code blocks by pressing play button in brackets on left
* Before you you start: change the runtime to "GPU" with "High RAM"
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param') 

References:  
* https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough
* https://www.tensorflow.org/tutorials/images/classification
* https://medium.com/analytics-vidhya/create-tensorflow-image-classification-model-with-your-own-dataset-in-google-colab-63e9d7853a3e
* https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_image_retraining.ipynb#scrollTo=umB5tswsfTEQ
* https://medium.com/analytics-vidhya/how-to-do-image-classification-on-custom-dataset-using-tensorflow-52309666498e
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
* https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

## Installs & Imports   
---

In [ ]:
#@title Choose where to save results & set up directory structure
# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]
print("Saving results ", save)

# Mount google drive to export image cropping coordinate file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your working directory in form field to right
import os
cwd = "/content/drive/MyDrive/train/tf2" #@param ["/content/drive/MyDrive/train/tf2"] {allow-input: true}
if not os.path.exists(cwd):
    os.makedirs(cwd)

# Folder where train/test images will be saved
train_folder = "images" #@param ["images"] {allow-input: true}
train_wd = cwd + '/pre-processing/' + train_folder
if not os.path.exists(train_wd):
    os.makedirs(train_wd)
print("\nTraining images directory set to: \n", train_wd)

# Folder where trained models will be saved
saved_models_folder = "saved_models" #@param ["saved_models"] {allow-input: true}
saved_models_wd = cwd + '/' + saved_models_folder
if not os.path.exists(saved_models_wd):
    os.makedirs(saved_models_wd)
print("\nTrained models directory set to: \n", train_wd)

# Folder where model training graphs will be saved
train_graphs_folder = "train_graphs" #@param ["train_graphs"] {allow-input: true}
train_graphs_wd = cwd + '/' + train_graphs_folder
if not os.path.exists(train_graphs_wd):
    os.makedirs(train_graphs_wd)
print("\nModel training graphs directory set to: \n", train_graphs_wd)

In [ ]:
# For working with data
import numpy as np
import pandas as pd
import os
import csv
import itertools

# For measuring inference time
import time

# For downloading and displaying images
import matplotlib
import matplotlib.pyplot as plt
from PIL import ImageFile, Image

# For image classification and training through TF-Hub
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Print Tensorflow version
print('Tensorflow Version: %s' % tf.__version__)

# Check available GPU devices
print('The following GPU devices are available: %s' % tf.test.gpu_device_name())

## Model & Training Dataset Preparation
---
Run these blocks every time you train to choose model hyperparameters and training dataset pre-processing steps.

In [ ]:
#@title Define functions

# Set checkpoint paths for training
def set_ckpt_path(saved_models_wd, resume_train_ckpt, TRAIN_SESS_NUM, num_epochs):
    # If resuming from a previous training attempt (= saved checkpoint)
    if resume_train_ckpt:
        TRAIN_SESS_NUM = TRAIN_SESS_NUM
        CKPTS_PATH = saved_models_wd + TRAIN_SESS_NUM + '/ckpt/'
        CKPT_PATH = CKPTS_PATH + 'cp-{epoch:04d}.ckpt'
        latest = tf.train.latest_checkpoint(CKPTS_PATH)
        print("Restoring weights from Model {}, Checkpoint {}".format(TRAIN_SESS_NUM, latest))
        model.load_weights(latest)
    # If new training attempt
    else:
        # Save each new training attempt results in new folder
        last_attempt = !ls $saved_models_wd | tail -n 1
        if not last_attempt:
            last_attempt = 0
        # Name folder to sort by attempt number (useful if many training runs)
        else:
            last_attempt = int(last_attempt.n)
            if last_attempt < 9:
                TRAIN_SESS_NUM = "0" + str(last_attempt + 1)
            else:
                TRAIN_SESS_NUM = str(last_attempt + 1)
    # Set checkpoint naming scheme
    CKPT_PATH = saved_models_wd + TRAIN_SESS_NUM + '/ckpt/cp-{epoch:04d}.ckpt' 
    print("Last training attempt number:", last_attempt)
    print("Training attempt number: {}, for {} epochs".format(TRAIN_SESS_NUM, num_epochs))

    return CKPT_PATH, last_attempt, TRAIN_SESS_NUM

# Set checkpoint callbacks for training
def set_ckpt_callbacks(CKPT_PATH):
    # Save weights for 0th epoch
    model.save_weights(CKPT_PATH.format(epoch=0))
    # Create a callback that saves the model's weights during training
    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CKPT_PATH,
                                                       save_weights_only=True,
                                                       verbose=1) # Verbosity mode: 0 = silent, 1 = progress bar, 2 = one line per epoch
    
    return ckpt_callback

# Plot loss and accuracy for training session
def plot_train_graph(train_graphs_dir, TRAIN_SESS_NUM, hist, test_generator):
    # Plot loss
    plt.figure()
    plt.title("Attempt {}: Training and Validation Loss".format(TRAIN_SESS_NUM))
    plt.xlabel("Training Steps")
    plt.ylim([0,2])
    plt.plot(hist["loss"], label='Train')
    plt.plot(hist["val_loss"], label='Test')
    plt.legend(loc='lower right')
    # Plot accuracy
    plt.figure()
    plt.title("Attempt {}: Training and Validation Accuracy".format(TRAIN_SESS_NUM))
    plt.xlabel("Training Steps")
    plt.ylim([0,1])
    plt.plot(hist["accuracy"], label='Train')
    plt.plot(hist["val_accuracy"], label='Test')
    plt.legend(loc='upper right')
    train_graph_path = train_graphs_dir + TRAIN_SESS_NUM + '.png'
    plt.savefig(train_graph_path)
    # Print final loss and accuracy values
    final_loss, final_accuracy = model.evaluate(test_generator, steps = test_steps)
    print('Final loss: {:.2f}'.format(final_loss))
    print('Final accuracy: {:.2f}%'.format(final_accuracy * 100))

In [ ]:
#@title Choose model type and batch size

# Use pre-trained model or build one from scratch?
use_pretrained_model = True #@param {type:"boolean"}

# Set model info
# To use pretrained model
if use_pretrained_model: 
    # Select pre-trained model to use from Tensorflow Hub Model Zoo
    module_selection = ("mobilenet_v2_1.0_224", 224) #@param ["(\"mobilenet_v2_1.0_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
    handle_base, pixels = module_selection
    IMAGE_SIZE = (pixels, pixels)
    if handle_base == "inception_v3":
        MODULE_HANDLE ="https://tfhub.dev/google/imagenet/inception_v3/classification/4".format(handle_base)
        epsilon = 1
    elif handle_base == "mobilenet_v2_1.0_224":
        MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4".format(handle_base) 
        epsilon = 1e-07
# To build model from scratch
else: 
    # Set input image size for model
    module_selection = ("from_scratch", 150)
    epsilon = 1

# Adjust batch size to make training faster or slower
BATCH_SIZE = "16" #@param ["16", "32", "64", "128"]

print("Using {} with input size {} and batch size {}".format(handle_base, IMAGE_SIZE, BATCH_SIZE))

In [ ]:
#@title Prepare training dataset *(Note: If output says line of output says "Found 0 images belonging to..." run again until a non-zero number is given. Some images throw errors in ImageDataGenerator but are fine for training)*

# To suppress warnings from pillow about image sizes
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 95000000

# Set path to folder where training images were stored in rating_preprocessing.ipynb
print("Loading training images from: \n", train_wd)

# Adjust interpolation method and see how training results change
interpolation = "nearest" #@param ["nearest", "bilinear"]

# Set data generation and flow parameters
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=int(BATCH_SIZE),
                       interpolation = interpolation, color_mode='rgb')

# Make test dataset
test_datagen = ImageDataGenerator(**datagen_kwargs)
test_generator = test_datagen.flow_from_directory(train_wd, 
                                                  subset="validation",
                                                  shuffle=True, **dataflow_kwargs)

# Make train dataset
train_datagen = ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(train_wd, 
                                                    subset="training", shuffle=True, 
                                                    **dataflow_kwargs)

# Learn more about data batches
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print("Dataset label classes: \n", dataset_labels)

### Prepare Model
If fine-tuning a pre-trained model, run first block only. If building custom model from scratch, run second block only.

In [ ]:
#@title Fine-tune pretrained model

# If model is overfitting, add/increase dropout rate
dropout_rate = 0.1 #@param {type:"slider", min:0, max:0.5, step:0.1}
lr = 0.001 #@param ["0.1", "0.01", "0.001", "0.0001", "0.00001"] {type:"raw"}

# reeze or unfreeze lower layers for transfer learning and fine tuning
## False freezes lower layers so only top classifier is retrained
trainable = True #@param ["True", "False"] {type:"raw"} 

# Build model
print("Building model with", handle_base)
def create_model():
    model = tf.keras.Sequential([InputLayer(input_shape=IMAGE_SIZE + (3,)),
                                 hub.KerasLayer(MODULE_HANDLE, trainable=trainable),
                                 Dropout(rate = dropout_rate),
                                 Dense(train_generator.num_classes,
                                       kernel_regularizer=tf.keras.regularizers.l2(0.0001))])
  
    # Build model
    model.build((None,)+IMAGE_SIZE+(3,))
  
    # Compile model
    model.compile(
        # Parameters for Adam optimizer
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, 
                                           beta_2=0.999, epsilon=epsilon, 
                                           amsgrad=False, name='Adam'), 
                  # Categorical cross entropy because 5 exclusive classes
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, 
                                                               label_smoothing=0),
                  metrics=['accuracy'])
    return model

# Create new model instance
model = create_model()

# Steps per epoch and testing
steps_per_epoch = train_generator.samples // train_generator.batch_size
test_steps = test_generator.samples // test_generator.batch_size

# Display model architecture
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
#@title Build model from scratch

# Adjust model perforamance using below hyperparameters
# See blog post for explanations https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

# Adjust learning rate
lr = 0.001 #@param ["0.1", "0.01", "0.001", "0.0001", "0.00001"] {type:"raw"}

# If model is overfitting, add/increase dropout rate
dropout_rate = 0.2 #@param {type:"slider", min:0, max:0.7, step:0.1}

# Layer 1: Start with smaller number of filters and increase number if performance too low
no_filters_lay1 = 64 #@param ["32", "64", "128"] {type:"raw"}

# Layer 2: Use either the same number of layers as Layer 1, or 2x as many
no_filters_lay2 = no_filters_lay1 * 2 #@param ["no_filters_lay1", "no_filters_lay1 * 2"] {type:"raw"}

# Layer 3: Use 2x as many layers as Layer 2
no_filters_lay3 = no_filters_lay2 * 2

# Layer 1: If input image size >128, may need to use initial filter size of 5, 5
filter_size_lay1 = (5, 5) #@param ["(3, 3)", "(5, 5)", "(7, 7)"] {type:"raw"}

# Compile model: Choose loss function. Categorical supposed to be better for multiple classes, but binary got better results one run
loss_fun = "categorical_crossentropy" #@param ["categorical_crossentropy", "binary_crossentropy"]
print("Building model from scratch")
def create_model():
  model = Sequential([
    Conv2D(no_filters_lay1, filter_size_lay1, padding='same', activation='relu',
        input_shape=(IMAGE_SIZE + (3,))),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(no_filters_lay2, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(no_filters_lay3, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(), # this converts our 3D feature maps to 1D feature vectors
    Dense(64, activation='relu'),
    Dropout(dropout_rate),
    Dense(train_generator.num_classes, activation='softmax') # softmax good for multiple class models with exclusive classes
])

  # Compile model
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon, amsgrad=False,
    name='Adam')
  model.compile(loss=loss_fun,
              optimizer=optimizer,
              metrics=['accuracy'])
  return model

# Create model instance
model = create_model()

# Set steps per epoch and testing
steps_per_epoch = train_generator.samples // train_generator.batch_size
test_steps = test_generator.samples // test_generator.batch_size

# Display model architecture
model.summary()

## Train
---

In [ ]:
#@title Train the model

# Adjust number of epochs to find balance between underfit and overfit for training
num_epochs = '10' #@param {type:"string"}

# Resume training from previous training attempt/checkpoint ?
resume_train_ckpt = True #@param {type:"boolean"}
# (Optional: Only if above is True) 
# Resume from which saved checkpoint?
if resume_train_ckpt:
    TRAIN_SESS_NUM = "13" #@param {type:"string"}


# Set where trained models will be saved
CKPT_PATH, last_attempt, TRAIN_SESS_NUM = set_ckpt_path(saved_models_wd, 
                                                        resume_train_ckpt, 
                                                        TRAIN_SESS_NUM,
                                                        num_epochs)

# Set up checkpoint callbacks for saving during training 
ckpt_callback = set_ckpt_callbacks(CKPT_PATH)

# Train the model with the new callback
hist = model.fit(train_generator,
                 epochs=int(num_epochs), steps_per_epoch=steps_per_epoch,
                 callbacks=[ckpt_callback],
                 validation_data=test_generator,
                 validation_steps=test_steps).history

# Save trained model 
saved_model_path = saved_models_wd + TRAIN_SESS_NUM
tf.keras.models.save_model(model, saved_model_path)

# Plot loss and accuracy for training session
plot_train_graph(train_graphs_wd, TRAIN_SESS_NUM, hist, test_generator)

## Review training results
---   
Display classification results on images for specific label classes

In [ ]:
#@title Define functions & parameters

# Display results for the most recently trained model?
use_last_attempt = True #@param {type:"boolean"}
# (Optional: Only if above is False) TO DO: Resume from which saved checkpoint?
TRAIN_SESS_NUM = "20" #@param ["13", "11", "15"] {allow-input: true}

# Only need to set parameters if not running images through last attempt
if not use_last_attempt:
    # Select model type for train attempt number
    module_selection = ("inception_v3", 299) #@param ["(\"mobilenet_v2_1.0_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
    # Label names
    dataset_labels = ["map", "phylo", "herb", "illus"] #@param ["[\"map\", \"phylo\", \"herb\", \"illus\"]"] {type:"raw", allow-input: true}

# Choose which image class to inspect results for
true_imclass = "map" #@param ["map", "phylo", "illus", "herb", "null"]

# Get test image paths
PATH_TO_TEST_IMAGES_DIR = train_wd + '/' + true_imclass
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]

# Load saved model from directory
def load_saved_model(use_last_attempt, saved_models_wd, TRAIN_SESS_NUM, module_selection):
    # If using last training attempt, get number from director
    if use_last_attempt:
        # Display results from most recent training attempt
        last_attempt = !ls $saved_models_wd | tail -n 1
        TRAIN_SESS_NUM = str(last_attempt.n)
    # Load trained model from path
    saved_model_path = saved_models_wd + TRAIN_SESS_NUM
    model = tf.keras.models.load_model(saved_model_path)
    # Get name and image size for model type
    handle_base, pixels = module_selection

    return model, pixels

# Define start and stop indices in EOL bundle for running inference   
def set_start_stop(run):
    # To test with a tiny subset, use 5 random bundle images
    if "tiny subset" in run:
        N = len(TEST_IMAGE_PATHS)
        start=np.random.choice(a=N, size=1)[0]
        stop=start+5
    # To run for up to 50 images
    else:
        start=np.random.choice(a=N, size=1)[0]
        stop=start+50
    
    return start, stop

# For reading an image from filename
def filename_to_image(fn):
    img = Image.open(im_path)
    disp_img = img.convert('RGB')
    inf_img = disp_img.resize((pixels, pixels))
    inf_img = np.reshape(inf_img,[1,pixels,pixels,3])
    image = inf_img*1./255

    return image, disp_img

# Get info from predictions to display on images
def get_predict_info(predictions, im_num, stop, start):
    # Get info from predictions
    label_num = np.argmax(predictions[0], axis=-1)
    conf = predictions[0][label_num]
    im_class = dataset_labels[label_num]
    # Display progress message after each image
    print('Inference complete for {} of {} images'.format(im_num, (stop-start)))

    return label_num, conf, im_class

In [ ]:
#@title Run inference

# Load saved model
model, pixels = load_saved_model(use_last_attempt, saved_models_wd, 
                                 TRAIN_SESS_NUM, module_selection)

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for all images"]
print("Run: ", run)

# Run EOL images through trained model and display results
start, stop = set_start_stop(run)
for im_num, im_path in enumerate(TEST_IMAGE_PATHS[start:stop], start=1):
    # Load in image
    image, disp_img = filename_to_image(im_path)
    
    # Image classification
    start_time = time.time() # Record inference time
    predictions = model.predict(image, batch_size=1)
    label_num, conf, im_class = get_predict_info(predictions, im_num, stop, start)
    end_time = time.time()

    # Display classification results with images
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(disp_img)
    plt.title('{}) Prediction: {}, Confidence: {}, Inference time: {}'.format(
              im_num, im_class, format(conf, '.2f'), format(end_time-start_time, '.2f')))